# Cleanup

## AutoReload

In [24]:
%load_ext autoreload
%autoreload 2

In [55]:
import os
import time
import boto3
from utils.s3 import s3_handler
from utils.ssm import parameter_store

In [33]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

## S3

In [34]:
s3 = s3_handler(region_name=pm.get_params(key=strPrefix + "REGION"))

This is a S3 handler with [ap-northeast-2] region.


In [42]:
strDataBucketName = "sagemaker-ap-northeast-2-419974056037"

In [43]:
s3.delete_bucket(bucket_name=strDataBucketName) ## delete bucket as well as objects

DELETE: [sagemaker-ap-northeast-2-419974056037] Bucket was deleted successfully


True

## pipeline

In [45]:
nModels = 30

In [47]:
for nIdx in range(nModels):
    strPipelineName = f"train-pipeline-model-{nIdx+1}"
    !aws sagemaker delete-pipeline --pipeline-name $strPipelineName 


An error occurred (ResourceNotFound) when calling the DeletePipeline operation: Pipeline 'arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-1' does not exist.

An error occurred (ResourceNotFound) when calling the DeletePipeline operation: Pipeline 'arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-2' does not exist.

An error occurred (ResourceNotFound) when calling the DeletePipeline operation: Pipeline 'arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-3' does not exist.

An error occurred (ResourceNotFound) when calling the DeletePipeline operation: Pipeline 'arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-4' does not exist.

An error occurred (ResourceNotFound) when calling the DeletePipeline operation: Pipeline 'arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-5' does not exist.

An error occurred (ResourceNotFound) when calling the DeletePipeline 

## Model registry

In [49]:
sagemaker_client = boto3.client('sagemaker')

In [56]:
def empty_and_delete_model_package(sagemaker_client, mpg_name):
    mpg = sagemaker_client.list_model_packages(
        ModelPackageGroupName=mpg_name,
    )
    
    # Delete model packages if Group not empty
    model_packages = mpg.get('ModelPackageSummaryList')
    if model_packages:
        for mp in model_packages:
            sagemaker_client.delete_model_package(
                ModelPackageName=mp['ModelPackageArn']
            )
            time.sleep(1)

    # Delete model package group
    sagemaker_client.delete_model_package_group(
        ModelPackageGroupName=mpg_name
    )

In [57]:
nModels = 30

In [58]:
for nIdx in range(nModels):
    DeleteModelGroupName = f"model-{nIdx+1}"
    empty_and_delete_model_package(sagemaker_client, DeleteModelGroupName)

## Parameter store

In [59]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + ["PREFIX"]
pm.delete_param(listParams)

  parameters: ['SM-PIPELINE-MULTI-MODELS-ACCOUNT-ID', 'SM-PIPELINE-MULTI-MODELS-DEFAULT-BUCKET', 'SM-PIPELINE-MULTI-MODELS-REGION', 'SM-PIPELINE-MULTI-MODELS-SAGEMAKER-ROLE-ARN', 'PREFIX'] is deleted successfully
